# 决策树

决策树和svm一样，既可以做分类也可以做回归，在这里主要讨论做分类的情况，即分类决策树模型，顾名思义，它是一个树模型，它的边一般表示条件，它的节点分为两类，一个是内部节点(非叶子节点)，一般表示一个特征，另一个是叶子节点表示一个类别。分类树模型的学习目标是从训练数据集中归纳出一组分类规则，当然这样的规则或许有无穷多个，又或许一个也没有，当规则有无穷多的时候怎么办？当然是优中选优了，我们需要的不仅是训练场上的好兵，更是在真正的战场上表现出色的将士，毕竟练兵的目的就是为了打仗。所以我们需要的是在训练集中表现出色且在测试集中性能良好的模型，这就要求模型要具备良好的泛化能力，暂且放下泛化这个话题，毕竟首要的目标是找到一个在训练集中表现出色的模型，决策树的学习算法是这样的: 递归的选择一个特征，并根据该特征对训练数据集(或其子数据集)进行分割。这句话看来看去发现我们也只能在"选择一个特征"这个方向上做努力了。这个就是特征选择问题了。

### 特征选择
特征选择的目的是选取对于训练数据有分类能力的特征，也即使用哪个特征来划分特征空间，当然，我们希望这个特征的分类能力超棒。通常是根据该特征分类前和分类后的信息增益或信息增益比以及基尼系数来选择特征。

## 决策树的生成
决策树的生成其实就是根据特征对训练数据集进行分类的过程，下面介绍的这几种算法主要的区别在于特征选择的方法不同

* ID3: 在决策树的各个节点上应用信息增益准则选择特征，递归地构造决策树。 方法: 从根节点开始，对节点计算所有特征的信息增益，选择信息增益最大的特征作为节点特征。由该特征的不同取值建立子节点，然后对子节点递归以上操作。直到所有的信息增益都很小或者没有特征可选时终止。

* C4.5: 在生成的过程中使用信息增益比来选择特征，其余同ID3相同。

* CART算法: 是应用最广泛的决策树学习方法，其生成的决策树为 分类与回归树(classification and regression tree, CART),与上述生成的决策树不同的是，CART既可以做分类也可以做回归，当CART是分类树时，采用GINI值作为节点分裂的依据；当CART是回归树时，采用样本的最小方差作为节点分裂的依据；CART是一个二叉树， 那怎么分裂成二叉树呢，因为我们知道，对于离散型属性，理论上有多少个离散值就应该分裂成多少个节点。但只须将其中一个离散值独立作为一个节点，其他的离散值生成另外一个节点即可。对于连续型属性，通过样本方差寻找最优分割点，将节点一分为二。

## 剪枝
通过上面的几个算法，我们已经能够找到在训练机上表现出色的模型了，接下来就该考虑泛化的问题了，在决策树模型中，由于决策树的建立完全是依赖于训练样本，因此该决策树对训练样本能够产生完美的拟合效果。但这样的决策树对于测试样本来说过于庞大而复杂，可能产生较高的分类错误率。这种现象就称为过拟合。因此需要将复杂的决策树进行简化，即去掉一些节点解决过拟合问题，这个过程称为剪枝。 

剪枝方法分为预剪枝和后剪枝两大类。预剪枝是在构建决策树的过程中，提前终止决策树的生长，从而避免过多的节点产生。预剪枝方法虽然简单但实用性不强，因为很难精确的判断何时终止树的生长。后剪枝是在决策树构建完成之后，对那些置信度不达标的节点子树用叶子结点代替，该叶子结点的类标号用该节点子树中频率最高的类标记。后剪枝方法又分为两种，一类是把训练数据集分成树的生长集和剪枝集；另一类算法则是使用同一数据集进行决策树生长和剪枝。常见的后剪枝方法有CCP(Cost Complexity Pruning)、REP(Reduced Error Pruning)、PEP(Pessimistic Error Pruning)、MEP(Minimum Error Pruning)。

#### 决策树代码实践

In [2]:
# sklearn中用的是CART算法
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

# 1. 加载数据
iris = load_iris()

X = iris.data[:, 2:] # length and width
y = iris.target

# 2. 构造模型
t_clf = DecisionTreeClassifier(max_depth=2)

# 3. 训练模型, 
t_clf.fit(X, y) # 决策树的众多特性之一就是，它不需要太多的数据预处理，尤其是不需要进行特征的缩放或者归一化
# 在树中的每个节点有三个比较重要的属性
# 节点的samples属性统计出它应用于多少个训练样本实例
# 节点的value属性告诉你这个节点对于每一个类别的样例有多少个
# 最后，节点的Gini属性用于测量它的纯度：如果一个节点包含的所有训练样例全都是同一类别的，我们就说这个节点是纯的

# 4. 预测
print(t_clf.predict([[5, 1.5]])) # 估计类别
print(t_clf.predict_proba([[5, 1.5]])) # 决策树还可以估计某个实例属于特定类k的概率

[1]
[[0.         0.90740741 0.09259259]]


#### 正则化超参数
决策树几乎不对训练数据做任何假设（于此相反的是线性回归等模型，这类模型通常会假设数据是符合线性关系的）如果不添加约束，树结构模型通常将根据训练数据调整自己，使自身够很好的拟合数据，而这种情况下大多数会导致模型过拟合。这一类的模型通常会被称为非参数模型，这不是因为它没有任何参数（通常也有很多），而是因为在训练之前没有确定参数的具体数量，所以模型结构可以根据数据的特性自由生长。于此相反的是，像线性回归这样的参数模型有事先设定好的参数数量，所以自由度是受限的，这就减少了过拟合的风险（但是增加了欠拟合的风险）。

DecisionTreeClassifier类还有一些其他的参数用于限制树模型的形状， 以此来增加模型的泛化能力

* min_samples_split: 节点在被分裂之前必须具有的最小样本数, 也就是说小于这个数目的节点不能再分裂了
* min_samples_leaf: 叶节点必须具有的最小样本数, 小于这个值的不能够被划分为一个类别
* max_leaf_nodes: 叶节点的最大数量
* max_features: 在每个节点被评估是否分裂的时候，具有的最大特征数量
* 增加min_* hyperparameters或者减少max_*hyperparameters会使模型正则化

### 回归
决策树也能够执行回归任务，使用Scikit-Learn的DecisionTreeRegressor类构建一个回归树

In [4]:
from sklearn.tree import DecisionTreeRegressor

# 获取数据
# Quadratic training set + noise
np.random.seed(42)
m = 200
X = np.random.rand(m, 1)
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X, y)

x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
y_pred = tree_reg.predict(x1)
# 它不是预测每个节点中的样本所属的分类，而是预测一个具体的数值
# CART算法的工作方式与之前处理分类模型基本一样，不同之处在于，现在不再以最小化不纯度的方式分割训练集，而是试图以最小化MSE的方式分割训练集

## 不稳定性
易于使用且功能丰富而强大。然而，它也有一些限制，首先，你可能已经注意到了，决策树很喜欢设定正交化的决策边界，（所有边界都是和某一个轴相垂直的），这使得它对训练数据集的旋转很敏感,更加通俗的讲,决策时的主要问题是它对训练数据的微小变化非常敏感